In [1]:
conda install selenium

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as numpy
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import datetime

def lazyloading(driver):
    # Get the current number of rows
    current_rows_number = len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0'))
    total_number = int(driver.find_elements_by_css_selector('.querySummary__querySummary--39WP2')[0].text.split('Showing ')[1].split('wines')[0])
    print("Total Number: ", total_number)
    while current_rows_number < total_number:
        # Scroll down to make new XHR (request more table rows)
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        try:
            driver.set_window_position(0,-1000)
            # Wait until number of rows increased       
            wait(driver, 300).until(lambda drive: len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0')) > current_rows_number)
            # Update variable with current rows number
            current_rows_number = len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0'))
            if current_rows_number % 200 == 0:
              print("Curr Row Number: ", current_rows_number)
        # If number of rows remains the same after 5 seconds passed, break the loop
        # as there no more rows to receive
        except TimeoutException:
            break

In [3]:
%reload_ext autoreload
%autoreload 2

In [7]:
from wine import Wine

australia_wine_url = "https://www.vivino.com/explore?e=eJwNyzEKgDAMRuHbZK6D4795A3ESkVhjKdhWkip6e7u85eMlRUcpZjhK_KJ3jvyHaSTfMtDVNBx4WKNUPqlsUK4xB1v5EeUgVLCL-XbZvIBvMnj-Ab-WHTk="
australia_wine = Wine(australia_wine_url)
australia_wine.setup_driver()
australia_wine.run_timestamp()
lazyloading(australia_wine.driver)
australia_wine.run_timestamp()

WebDriverException: Message: unknown error: cannot determine loading status
from unknown error: cannot determine loading status
from disconnected: received Inspector.detached event
  (Session info: chrome=79.0.3945.130)


In [ ]:
australia_html = BeautifulSoup(australia_wine.driver.page_source, 'lxml')
australia_div = australia_html.find("div", {"class": "explorerPage__results--3wqLw"})
australia_rows = australia_html.find_all("div", {"class": "explorerCard__explorerCard--3Q7_0"})

In [ ]:
import copy

# austria_wine_df = pd.DataFrame()
all_rows = []

# Let's store each row as a dictionary 
empty_row = {
    "title": None, "country": None, "continent": None, "ratings": None, "num_ratings": None, "reviews": None, "image_url": None
}

for row in australia_rows:
    new_row = copy.copy(empty_row)
    # A list of all the entries in the row.
    new_row['title'] = row.find("span", {"class": "vintageTitle__wine--U7t9G"}).text
    new_row['country'] = row.find("i", {"class": "vintageLocation__countryFlag--1HbXr"})['title']
    new_row['continent'] = 'Oceania'
    new_row['ratings'] = row.find("div", {"class": "vivinoRatingWide__averageValue--1zL_5"}).text
    new_row['num_ratings'] = int(row.find("div", {"class": "vivinoRatingWide__basedOn--s6y0t"}).text.split()[0])
    review_div = row.find("div", {"class": "review__note--2b2DB"})
    if review_div:
        new_row['review'] = review_div.text
    image_div = row.find("div", {"class": "cleanWineCard__bottleShotWrapper--nymTj"})
    if image_div:
        new_row['image_url'] = image_div.find("div").find_next('img')['src'].strip("//")
    all_rows.append(new_row)

In [ ]:
all_rows[0]